# An fMRI study of composition in noun and verb phrases
### Bonnasse-Gahot, Bemis, Perez-Guevara, Dehaene, Pallier
Generate the events for the incremental model

In [1]:
import numpy as np
import pandas as pd
import os.path as op
import simpcomp as sc

In [2]:
# regressors are:
# cs : consonant string
# wn1 : first word in a noun sequence
# wnp : word added to a noun phrase
# wnl : word added to a noun list
# wv1 : first word in a verb sequence
# wvp : word added to a verb phrase
# wvl : word added to a verb list
# probe
# resp_r : response key right
# resp_l : response key left 

def create_model_events(full_events):
    model_events = pd.DataFrame(columns=['onset', 'duration', 'trial_type']) 

    is_first_word = True
    for idx, event in full_events.iterrows():    
        if event.stim.isupper(): # probe        
            model_events = model_events.append(pd.Series({'onset':event.onset,
                                                          'duration':0.0,
                                                          'trial_type':'probe'}),
                                               ignore_index=True)
        elif event.stim == '[response]':
            if event.response_key == 'y':
                trial_type = 'resp_r'
            else:
                trial_type = 'resp_l'
            model_events = model_events.append(pd.Series({'onset':event.onset,
                                                          'duration':0.0,
                                                          'trial_type':trial_type}),
                                               ignore_index=True)
        # check whether it's a consonant string or a word
        # cheap here -- simply look whether it contains at least one vowel
        elif not np.any([letter_tmp in 'aeiouyàèùéâêîôûëïüÿ' for letter_tmp in event.stim]): # consonant string
        #sum([letter_tmp in 'aeiouyàèùéâêîôûëïüÿ' for letter_tmp in event.stim])>0:
            model_events = model_events.append(pd.Series({'onset':event.onset,
                                                          'duration':event.duration,
                                                          'trial_type':'cs'}),
                                               ignore_index=True)
            is_first_word = True # reinitialize is_first_word, as sequences always begin with a consonant string
        elif is_first_word:
            model_events = model_events.append(pd.Series({'onset':event.onset,
                                                          'duration':event.duration,
                                                          'trial_type':'w{}1'.format(event.trial_type[0])}),
                                               ignore_index=True)
            is_first_word = False
        else:
            model_events = model_events.append(pd.Series({'onset':event.onset,
                                                          'duration':event.duration,
                                                          'trial_type':'w{}{}'.format(event.trial_type[0],
                                                                                      event.trial_structure[0])}),
                                               ignore_index=True)
    return model_events

In [3]:
for sub_id in sc.subject_list:
    dir_tmp = op.join(sc.model_incremental_folder, 'events', sub_id)
    sc.make_dir(dir_tmp)
    for indrun in range(1,5):
        filename_in = op.join(sc.events_folder, sub_id, 'func',
                   '{}_task-maindesign_run-{:02d}_events.tsv'.format(sub_id, indrun))
        full_events = pd.read_csv(filename_in, sep='\t')
        full_events.drop(full_events[full_events.stim=='+'].index, inplace=True)
        model_events = create_model_events(full_events)
        
        filename_out = op.join(dir_tmp, 
                      '{}_model-incremental_run-{:02d}_events.tsv'.format(sub_id, indrun))
        model_events.to_csv(filename_out, index=False, sep='\t')